In [2]:
!pip install langchain_qdrant qdrant_client langchain-openai langchain_openai langchain_community unstructured markdown

In [3]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
#os.environ["OPENAI_API_KEY"] = ""

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]="https://chechia-workshop.openai.azure.com/"
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="gpt-4.1-mini"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]="gpt-4.1-mini"
#os.environ["OPENAI_MODEL"]="text-embedding-3-small"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

AZURE_OPENAI_API_KEY is ready


In [20]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    temperature="0.0",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    #rate_limiter=InMemoryRateLimiter(
    #    requests_per_second=60,
    #    check_every_n_seconds=1,
    #)
)

openai_embeddings = AzureOpenAIEmbeddings(
    model='text-embedding-3-large',
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
)

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance
from qdrant_client.models import VectorParams

# initialize the qdrant client
client = QdrantClient(
    host="qdrant",
    prefer_grpc=True,
)

collection_name = "k8s-official-doc"

if not client.collection_exists(collection_name):
    client.create_collection(
       collection_name=collection_name,
       vectors_config=VectorParams(
           size=3072,
           distance=Distance.COSINE
        ),
    )
# create the vector store
vectordb = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=openai_embeddings,
)

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

chat_history = []

condense_question_system_template = """
    給定一段對話歷史和最新的用戶問題，用戶問題可能參考對話歷史中的內容，
    重新構造一個獨立的問題，該問題可以在沒有對話歷史的情況下理解。
    不要直接回答問題，只有在需要時重新構造問題，否則原樣返回。
"""

condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

system_prompt = """
    你是一個問答任務的助手。根據 Context 的資料內容回答問題。
    不要回答 Context 以外的資訊
    如果 Context 的資料沒有相關內容，就基於 Context 的內容做解釋。
    使用繁體中文回答。
    回答的格式為
        Question：問題
        Context: Context
        Answer: 答案

    Context:
    {context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

vb_retriever = vectordb.as_retriever(
    #search_type="similarity",
    search_type="mmr",
    search_kwargs={
        "k": 4,
        #"score_threshold": 0.5,
    }
)

# Create a history-aware retriever from the OpenAI model and the retriever
convo_qa_chain = create_retrieval_chain(
    create_history_aware_retriever(
        llm,
        vb_retriever,
        condense_question_prompt
    ),
    create_stuff_documents_chain(
        llm,
        qa_prompt
    )
)

In [22]:
def generate_prompt(question):
    if not chat_history:
        prompt = f"""
        問題：{question}
        """
    else:
        print("Chat with previous history")
        context_entries = [f"Question: {q}\nAnswer: {a}" for q, a in chat_history[-3:]]
        context = "\n\n".join(context_entries)
        prompt = f"""
            使用最近對話提供的內容，以簡潔且具信息性的方式回答新問題。
            最近對話的內容：{context}
            新問題：{question}
            答案：
        """

    return prompt

def ask_question(question):
    prompt = generate_prompt(question)
    # Invoke the chain with the question and the chat history
    result = convo_qa_chain.invoke(
        {
            "input": prompt,
            "chat_history": chat_history
        }
    )

    # Return the bot's answer
    return result["answer"]

In [23]:
print(ask_question("如何擴展 Service IP 範圍?"))

Question：如何擴展 Service IP 範圍?  
Context:   
Answer: 根據提供的 Context，沒有相關資料說明如何擴展 Service IP 範圍，因此無法直接回答此問題。建議參考相關系統或平台的官方文件，了解如何調整或擴展 Service IP 範圍的具體步驟。


In [24]:
documents = []

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
#from langchain_community.document_loaders import UnstructuredMarkdownLoader

# https://kubernetes.io/docs/tasks/extend-kubectl/kubectl-plugins/

loader = DirectoryLoader(
    "website/content/en/docs/tasks/network/",
    #"website/content/zh-cn/docs/tasks/network/",
    glob="./**/*.md",
    show_progress=True,
    recursive=True,
    loader_cls=TextLoader,
    #loader_cls=UnstructuredMarkdownLoader,
    #loader_kwargs={"mode":"single"},
    #loader_kwargs={"mode":"elements"},
)

documents = loader.load()
documents[0]

100%|██████████| 5/5 [00:00<00:00, 689.38it/s]


Document(metadata={'source': 'website/content/en/docs/tasks/network/extend-service-ip-ranges.md'}, page_content='---\nreviewers:\n- thockin\n- dwinship\nmin-kubernetes-server-version: v1.29\ntitle: Extend Service IP Ranges\ncontent_type: task\n---\n\n<!-- overview -->\n{{< feature-state feature_gate_name="MultiCIDRServiceAllocator" >}}\n\nThis document shares how to extend the existing Service IP range assigned to a cluster.\n\n\n## {{% heading "prerequisites" %}}\n\n{{< include "task-tutorial-prereqs.md" >}}\n\n{{< version-check >}}\n\n{{< note >}}\nWhile you can use this feature with an earlier version, the feature is only GA and officially supported since v1.33.\n{{< /note >}}\n\n<!-- steps -->\n\n## Extend Service IP Ranges\n\nKubernetes clusters with kube-apiservers that have enabled the `MultiCIDRServiceAllocator`\n[feature gate](/docs/reference/command-line-tools-reference/feature-gates/) and have the\n`networking.k8s.io/v1beta1` API group active, will create a ServiceCIDR objec

In [25]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"), 
    ("##", "Header 2"), 
    #("###", "Header 3"), 
    #("####", "Header 4")
]

text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    #strip_headers=False # keep headers in the text
)

all_sections = []

for doc in documents:
    sections = text_splitter.split_text(doc.page_content)

    for i in range(len(sections)):
        # keep metadata from the original document
        metadata = dict(doc.metadata)
        metadata.update(sections[i].metadata)
        metadata.update({"split": f"{i+1}/{len(sections)}"})
        sections[i].metadata = metadata
    #for section in sections:
    #    # keep metadata from the original document
    #    metadata = dict(doc.metadata)
    #    metadata.update(section.metadata)
    #    section.metadata = metadata

    all_sections.extend(sections)

print(f"=====Split {len(all_sections)} sections")
#print(f"=====First section: {all_sections[0]}")
#print(f"=====Last section: {all_sections[-1]}")

=====Split 18 sections


In [26]:
import hashlib

def add_document(document):
    # hash the source (url) to get a unique id
    print('Embedding progress:' + document.metadata['source'] + document.metadata['split'])
    hash = hashlib.sha256(
        str(document.metadata['source'] + document.metadata['split']).encode('utf-8')
    ).hexdigest()[::2]
    
    vectordb.add_documents(
        documents=[document],
        #ids=[str(uuid4())]
        ids=[hash]
    )

for section in all_sections:
    add_document(section)

collection_info = client.get_collection(collection_name=collection_name)
collection_info.points_count

Embedding progress:website/content/en/docs/tasks/network/extend-service-ip-ranges.md1/5
Embedding progress:website/content/en/docs/tasks/network/extend-service-ip-ranges.md2/5
Embedding progress:website/content/en/docs/tasks/network/extend-service-ip-ranges.md3/5
Embedding progress:website/content/en/docs/tasks/network/extend-service-ip-ranges.md4/5
Embedding progress:website/content/en/docs/tasks/network/extend-service-ip-ranges.md5/5
Embedding progress:website/content/en/docs/tasks/network/validate-dual-stack.md1/4
Embedding progress:website/content/en/docs/tasks/network/validate-dual-stack.md2/4
Embedding progress:website/content/en/docs/tasks/network/validate-dual-stack.md3/4
Embedding progress:website/content/en/docs/tasks/network/validate-dual-stack.md4/4
Embedding progress:website/content/en/docs/tasks/network/customize-hosts-file-for-pods.md1/4
Embedding progress:website/content/en/docs/tasks/network/customize-hosts-file-for-pods.md2/4
Embedding progress:website/content/en/docs

18

In [27]:
print(ask_question("如何擴展 Service IP 範圍?"))

Question：如何擴展 Service IP 範圍?  
Context:  
本文件分享如何擴展分配給 Kubernetes 叢集的 Service IP 範圍。  
透過設定 kube-controller-manager 的 `--service-cluster-ip-range` 參數，可以配置多個 IP 範圍，並利用 MultiCIDRServiceAllocator 功能來擴展現有的 Service IP 範圍。  
此外，建立 Service 時可透過 `.spec.ipFamilyPolicy` 和 `.spec.ipFamilies` 來指定使用的 IP 家族（IPv4、IPv6 或雙棧），以配合擴展後的多個 IP 範圍。  

Answer:  
要擴展 Service IP 範圍，可以在 kube-controller-manager 中設定多個 `--service-cluster-ip-range`，啟用 MultiCIDRServiceAllocator 功能，讓 Kubernetes 支援多個 IP 範圍。建立 Service 時，透過設定 `.spec.ipFamilyPolicy`（如 SingleStack、PreferDualStack）和 `.spec.ipFamilies`（指定 IPv4、IPv6 或雙棧）來使用擴展後的 IP 範圍，從而達成 Service IP 範圍的擴展與管理。


In [28]:
#client.delete_collection(collection_name=collection_name)